In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import re
from scipy import stats
import time


In [2]:
def get_paper_links_scrape(posters=0):
    # paper_link_CSS = 'ul > li > h4 > a:nth-child(1)'
    session = webdriver.Chrome()
    session.implicitly_wait(10)
    session.get('https://openreview.net/group?id=ICLR.cc/2021/Conference')
    paper_links = set()
    if posters == 1:
        session.find_element_by_css_selector('#notes > div > ul > li:nth-child(2) > a').click()
    elif posters == 2:
        session.find_element_by_css_selector('#notes > div > ul > li:nth-child(3) > a').click()

    header = 'accepted-oral-papers'
    if posters == 1:
        header = 'accepted-poster-papers'
    elif posters==2:
        header = 'rejected-papers'
    paper_link_CSS = f'#{header} > ul > li > h4 > a:nth-child(1)'
    prev_len = []
    for i in range(100):
        session.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        new_papers = set(session.find_elements_by_css_selector(paper_link_CSS))
        new_papers = set([i.get_attribute('href') for i in new_papers])
        paper_links = paper_links.union(new_papers)
        time.sleep(1)
        if len(prev_len) > 5 and len(paper_links) == prev_len[-5]:
            break
        prev_len.append(len(paper_links))
    new_papers = set(session.find_elements_by_css_selector(paper_link_CSS))
    new_papers = set([i.get_attribute('href') for i in new_papers])
    paper_links = paper_links.union(new_papers)
    session.close()
    return paper_links


In [3]:
def get_paper_links_load(filename):
    return [i['url'] for i in json.load(open(filename))]


In [5]:
paper_links = get_paper_links_scrape(
    2) | get_paper_links_scrape(1) | get_paper_links_scrape(0)
# paper_links = get_paper_links_load('data/iclr2018.json')


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 76


In [6]:


paper_links = list(paper_links)
papers = {}
print(len(paper_links))



NameError: name 'paper_links' is not defined

In [ ]:
session = webdriver.Chrome()
start_location = len(papers)

idx = start_location
while idx < len(paper_links):
    link = paper_links[idx]
    print(len(papers.items()), link)
    data = {}

    session.get(link)
    time.sleep(2)

    def update_non_comments():
        title = session.find_element_by_css_selector(
            'div.title_pdf_row > h2 > a:nth-child(1)').get_attribute('text')
        abstract = session.find_element_by_css_selector(
            '#content > div:nth-child(1) > div.note_contents > span.note_content_value').get_attribute('innerHTML')

        data['title'] = title
        data['abstract'] = abstract
        authors = session.find_element_by_css_selector(
            '.signatures').find_elements_by_css_selector('a')
        data['authors'] = [i.get_attribute('innerText') for i in authors]
        data['emails'] = [i.get_attribute(
            'data-original-title') for i in authors]
        data['emails'] = list(set([i[i.find('@')+1:] for i in data['emails']]))
        print(data['authors'], data['emails'])
        papers[link] = data

    comment_CSS = '#note_children > div.note_with_children'
    try:
        WebDriverWait(session, 30).until(
            lambda session: len(session.find_elements_by_css_selector(comment_CSS)) >= 1)
    except:
        if len(session.find_elements_by_css_selector('.spinner')) == 1:
            continue
        elif session.current_url == 'https://openreview.net/':
            continue
        elif '502' in session.title:
            continue
        elif '404' in session.title:
            idx += 1
            continue
        else:
            idx += 1
            update_non_comments()
            continue
    comments = session.find_elements_by_css_selector(comment_CSS)
    comment_vals = []
    for comment in comments:
        sections = comment.find_elements_by_css_selector('.note_contents')
        section_vals = {}
        for section in sections:
            field = section.find_element_by_css_selector('.note_content_field')
            field = field.get_attribute('innerHTML').strip()
            content = section.find_element_by_css_selector(
                '.note_content_value')
            content = content.get_attribute('innerHTML').strip()
            section_vals[field] = content
        comment_vals.append(section_vals)
    update_non_comments()
    data['comments'] = comment_vals

    papers[link] = data
    idx += 1


In [ ]:


print(len(papers))


In [ ]:
import copy
scraped_papers = copy.deepcopy(papers)
print(len(papers), idx)


In [ ]:


papers = copy.deepcopy(scraped_papers)
for paper in list(papers.values()):
    if 'comments' not in paper:
        continue
    for idx in range(len(paper['comments'])):
        comment = paper['comments'][idx]
        comment = {k.strip(': '): v for k, v in comment.items()}
        if 'Rating' in comment:
            comment['Rating'] = re.search(r'^\d+', comment['Rating']).group()
        if 'Confidence' in comment:
            comment['Confidence'] = re.search(
                r'^\d+', comment['Confidence']).group()
        else:
            comment['Confidence'] = '4'
        paper['comments'][idx] = comment


In [ ]:


def generate_top_papers(papers):
    overall_ratings = {}
    for paper in list(papers.values()):
        real_ratings = []
        ratings = []
        confidence = []
        if 'comments' not in paper:
            continue
        for comment in paper['comments']:
            if 'Rating' in comment:
                real_ratings.append(int(comment['Rating']))
                ratings.append(int(comment['Rating'])
                               * int(comment['Confidence']))
                confidence.append(int(comment['Confidence']))

        if len(ratings) == 0:
            continue
    #     print(paper['title'])
        overall_ratings[paper['title']] = (sum(
            ratings)/(len(ratings)*(sum(confidence)/len(confidence))), real_ratings, confidence)

    print(len(overall_ratings))
    all_ratings = sorted(list(overall_ratings.items()),
                         key=lambda x: x[1][0], reverse=True)
    for name, rating in all_ratings:
        print(
            f'{"%.2f" % rating[0]},{name},{tuple(rating[1])},{tuple(rating[2])}')


generate_top_papers(papers)


In [ ]:
final_json = []
review_scores = []
confidence_scores = []
for url, paper in papers.items():
    result = {}
    result['url'] = url
    real_ratings = []
    ratings = []
    confidence = []
    if 'comments' in paper:
        for comment in paper['comments']:
            if 'Rating' in comment and 'Confidence' in comment:
                real_ratings.append(int(comment['Rating']))
                ratings.append(int(comment['Rating'])
                               * int(comment['Confidence']))
                review_scores.append(int(comment['Rating']))
                confidence_scores.append(int(comment['Confidence']))
                confidence.append(int(comment['Confidence']))
            elif 'Decision' in comment:
                result['decision'] = comment['Decision']
            elif 'Recommendation' in comment:
                result['decision'] = comment['Recommendation']

        result['comments'] = paper['comments']
    result['ratings'] = real_ratings
    if len(ratings) != 0:
        result['rating'] = "%.2f" % (
            sum(ratings)/(len(ratings)*(sum(confidence)/len(confidence))))
    else:
        result['rating'] = ' N/A'
    result['confidences'] = confidence
    result['abstract'] = paper['abstract']
    result['title'] = paper['title']
    result['authors'] = paper['authors']
    result['emails'] = paper['emails']
    final_json.append(result)

sorted_final = sorted(final_json, key=lambda x: x['rating'], reverse=True)
final_json = []
for idx, i in enumerate(sorted_final):
    i['rank'] = idx+1
    final_json.append(i)

f = open('data.json', 'w')
print(f.write(json.dumps(final_json, indent=2)))
f.close()


In [ ]:
import numpy as np

f = open('data.json', 'r')
final_json = json.loads(f.read())
f.close()

ratings = [float(i['rating']) for i in final_json if i['rating'] != ' N/A']
ratings = sorted(ratings)
print(sum(ratings)/len(ratings))
print(ratings[len(ratings)*3//4])
# review_dict = {k:0 for k in range(1, 11)}
# for i in review_scores:
#     review_dict[i] += 1
# print(sum(review_scores)/len(review_scores))
# print(review_dict)
# confidence_dict = {k:0 for k in range(1, 6)}
# for i in confidence_scores:
#     confidence_dict[i] += 1
# print(confidence_dict)


def variance(arr, conf):
    if len(arr) == 0:
        return 0
    return np.var(arr)

#     mu = sum(arr)/len(arr)
#     var = 0
#     for idx in range(len(arr)):
#         var += (arr[idx] - mu)**2
#     )
#     return var/len(arr)


variance = [(i['title'], variance(i['ratings'], i['confidences']))
            for i in final_json]

variance = sorted(variance, key=lambda x: x[1], reverse=True)
variance_dict = {k: v for k, v in variance}
for title, v in variance[:10]:
    print(f'{"%.2f" % v}: {title}')

entropies = [(i['title'], stats.entropy(i['ratings'])/len(i['ratings']))
             for i in final_json]
entropies = sorted(entropies, key=lambda x: x[1])
print(entropies[:10])
for idx, i in enumerate(final_json):
    i['variance'] = float("%.2f" % variance_dict[i['title']]**(1/2))
    final_json[idx] = i

f = open('data.json', 'w')
print(f.write(json.dumps(final_json, indent=2)))
f.close()
# print(final_json)
